<a href="https://colab.research.google.com/github/santhosh201cali/fmml2021/blob/main/Lab2_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>
 Module Coordinator: Thrupthi Ann John thrupthi.ann@research.iiit.ac.in <br>
 Release date: 11 October 2021 Monday <br>

 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [4]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)
print("import successfully")

import successfully


In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. 

Let us download and examine the dataset. 

In [5]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


In [6]:
dataset.target[12]

2

Here is a function for calculating the 1-nearest neighbours

In [10]:
def NN1(traindata, trainlabel, query):#x,y , query
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here 
  # print(diff)
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data 
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel


We will also define a 'random classifier', which randomly allots labels to each sample

In [9]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. 

In [11]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [12]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [13]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [14]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [15]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Streaming output truncated to the last 5000 lines.
 [  2.6411       1.           3.81453076 ...   0.67155286   3.08
   -2.23      ]
 [  0.4889      -2.           2.08091554 ...   0.20599613   3.29
   -1.76      ]
 [  1.3172     -11.           2.29017797 ...   0.2623712    3.18
   -1.91      ]]
[[ 4.1121     29.          2.8045535  ... -0.37385952  1.65
  -2.93      ]
 [ 2.4978     29.          0.33376951 ... -0.6281742   1.65
  -2.94      ]
 [ 0.5138     29.         -0.55167547 ... -1.04771473  1.64
  -2.94      ]
 ...
 [ 0.5672      5.          1.29548768 ... -0.14995661  3.07
  -2.25      ]
 [-1.585       2.         -0.43812754 ... -0.61551334  3.28
  -1.78      ]
 [-0.7567     -7.         -0.22886511 ... -0.55913827  3.17
  -1.93      ]]
[[ 6.1612     41.          4.19548384 ...  0.05146116  1.66
  -2.92      ]
 [ 4.5469     41.          1.72469984 ... -0.20285352  1.66
  -2.93      ]
 [ 2.5629     41.          0.83925486 ... -0.62239405  1.65
  -2.93      ]
 ...
 [ 2.6163     17.  

For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [16]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Streaming output truncated to the last 5000 lines.
 [ 1.0333      4.          1.6258522  ...  0.75854277 -1.32
   0.86      ]
 [-1.1189      1.         -0.10776302 ...  0.29298604 -1.11
   1.33      ]
 [-0.2906     -8.          0.10149941 ...  0.34936111 -1.22
   1.18      ]]
[[ 3.1475     42.          1.45322588 ...  0.07139144 -2.72
   0.18      ]
 [ 1.5332     42.         -1.01755812 ... -0.18292324 -2.72
   0.17      ]
 [-0.4508     42.         -1.9030031  ... -0.60246377 -2.73
   0.17      ]
 ...
 [-0.3974     18.         -0.05583995 ...  0.29529435 -1.3
   0.86      ]
 [-2.5496     15.         -1.78945517 ... -0.17026238 -1.09
   1.33      ]
 [-1.7213      6.         -1.58019274 ... -0.11388731 -1.2
   1.18      ]]
[[ 3.57210000e+00  4.20000000e+01  1.57121252e+00 ... -4.62355498e-01
  -2.78000000e+00  1.00000000e-01]
 [ 1.95780000e+00  4.20000000e+01 -8.99571479e-01 ... -7.16670179e-01
  -2.78000000e+00  9.00000000e-02]
 [-2.62000000e-02  4.20000000e+01 -1.78501646e+00 ... -1.13

Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. 

Now let us try another random split and check the validation accuracy

In [17]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Streaming output truncated to the last 5000 lines.
 ...
 [ -0.2362     -16.           0.22997241 ...   0.15504138  -1.39
    0.87      ]
 [  0.9268     -15.           1.564016   ...   0.34874344  -1.31
    0.84      ]
 [ -0.3971     -27.           0.03966322 ...  -0.06043822  -1.21
    1.16      ]]
[[ 4.3877     24.          1.78047957 ... -0.37149612 -2.68
   0.22      ]
 [ 4.3639      4.          1.03448967 ... -0.81720984 -2.7
   0.23      ]
 [ 1.7056     35.          0.61370418 ... -0.37910647 -2.71
   0.2       ]
 ...
 [-1.388      10.          0.24137876 ... -0.09459094 -1.37
   0.92      ]
 [-0.225      11.          1.57542235 ...  0.09911112 -1.29
   0.89      ]
 [-1.5489     -1.          0.05106956 ... -0.31007054 -1.19
   1.21      ]]
[[ 4.2153     31.          0.14921727 ... -0.17531289 -2.69
   0.19      ]
 [ 4.1915     11.         -0.59677263 ... -0.62102662 -2.71
   0.2       ]
 [ 1.5332     42.         -1.01755812 ... -0.18292324 -2.72
   0.17      ]
 ...
 [-1.5604     1

You can run the above cell multiple times to try with different random splits. 
We notice that the accuracy is different for each run, but close together. 

Now let us compare it with the accuracy we get on the test dataset. 

In [18]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Streaming output truncated to the last 5000 lines.
 ...
 [ 6.17100000e-01  9.00000000e+00  1.69370391e+00 ...  3.81176341e-01
  -1.28000000e+00  8.50000000e-01]
 [-1.53510000e+00  6.00000000e+00 -3.99113082e-02 ... -8.43803893e-02
  -1.07000000e+00  1.32000000e+00]
 [-7.06800000e-01 -3.00000000e+00  1.69351127e-01 ... -2.80053178e-02
  -1.18000000e+00  1.17000000e+00]]
[[ 5.8402     21.          1.83706816 ...  0.10261438 -2.63
   0.17      ]
 [ 5.8164      1.          1.09107826 ... -0.34309935 -2.65
   0.18      ]
 [ 4.7724     32.          3.14107677 ...  0.34931871 -2.66
   0.16      ]
 ...
 [ 1.2275      8.          1.63201094 ...  0.57322161 -1.24
   0.84      ]
 [-0.9247      5.         -0.10160428 ...  0.10766488 -1.03
   1.31      ]
 [-0.0964     -4.          0.10765816 ...  0.16403996 -1.14
   1.16      ]]
[[ 5.61520000e+00  2.40000000e+01  1.73067076e+00 ...  3.06707629e-01
  -2.66000000e+00  1.20000000e-01]
 [ 5.59140000e+00  4.00000000e+00  9.84680861e-01 ... -1.39006098e-

### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced? 

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. 

In [19]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [20]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Streaming output truncated to the last 5000 lines.
 ...
 [ 6.17100000e-01  9.00000000e+00  1.69370391e+00 ...  3.81176341e-01
  -1.28000000e+00  8.50000000e-01]
 [-1.53510000e+00  6.00000000e+00 -3.99113082e-02 ... -8.43803893e-02
  -1.07000000e+00  1.32000000e+00]
 [-7.06800000e-01 -3.00000000e+00  1.69351127e-01 ... -2.80053178e-02
  -1.18000000e+00  1.17000000e+00]]
[[ 5.8402     21.          1.83706816 ...  0.10261438 -2.63
   0.17      ]
 [ 5.8164      1.          1.09107826 ... -0.34309935 -2.65
   0.18      ]
 [ 4.7724     32.          3.14107677 ...  0.34931871 -2.66
   0.16      ]
 ...
 [ 1.2275      8.          1.63201094 ...  0.57322161 -1.24
   0.84      ]
 [-0.9247      5.         -0.10160428 ...  0.10766488 -1.03
   1.31      ]
 [-0.0964     -4.          0.10765816 ...  0.16403996 -1.14
   1.16      ]]
[[ 5.61520000e+00  2.40000000e+01  1.73067076e+00 ...  3.06707629e-01
  -2.66000000e+00  1.20000000e-01]
 [ 5.59140000e+00  4.00000000e+00  9.84680861e-01 ... -1.39006098e-

This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?
